In [1]:
import numpy as np
import tensorflow 
import keras
import matplotlib.pyplot as plt

In [3]:
def define_discriminator(in_shape = (28, 28, 1)):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid', input_shape = in_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.MaxPool2D((3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [4]:
def define_generator(noise):
    gen_model = keras.Sequential()
    gen_model.add(keras.layers.Dense(150, input_dim = noise))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(250))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(450))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(600))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(700))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(784))
    gen_model.add(keras.activations.sigmoid())
    gen_model.add(keras.layers.Dropout(0.3))
    return gen_model

In [5]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = keras.Sequential()
    model.add(g_model)
    model.add(d_model)
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [6]:
def load_real_samples():
    (train_X, _), (_, _) = keras.datasets.mnist.load_data()
    train_X = np.expand_dims(train_X, axis = -1)
    train_X = train_X/255
    return train_X

In [7]:
def generate_real_samples(dataset, n_samples):
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    Y = np.ones((n_samples, 1))
    return X, Y